In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/factors-affecting-campus-placement/Placement_Data_Full_Class.csv


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,confusion_matrix
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols
import warnings
warnings.filterwarnings("ignore")
df1 = pd.read_csv('/kaggle/input/factors-affecting-campus-placement/Placement_Data_Full_Class.csv')
df=df1.copy(deep=True)
print(df)

     sl_no gender  ssc_p    ssc_b  hsc_p    hsc_b     hsc_s  degree_p  \
0        1      M  67.00   Others  91.00   Others  Commerce     58.00   
1        2      M  79.33  Central  78.33   Others   Science     77.48   
2        3      M  65.00  Central  68.00  Central      Arts     64.00   
3        4      M  56.00  Central  52.00  Central   Science     52.00   
4        5      M  85.80  Central  73.60  Central  Commerce     73.30   
..     ...    ...    ...      ...    ...      ...       ...       ...   
210    211      M  80.60   Others  82.00   Others  Commerce     77.60   
211    212      M  58.00   Others  60.00   Others   Science     72.00   
212    213      M  67.00   Others  67.00   Others  Commerce     73.00   
213    214      F  74.00   Others  66.00   Others  Commerce     58.00   
214    215      M  62.00  Central  58.00   Others   Science     53.00   

      degree_t workex  etest_p specialisation  mba_p      status    salary  
0     Sci&Tech     No     55.0         Mkt&HR 

In [3]:
df.isnull().sum()

sl_no              0
gender             0
ssc_p              0
ssc_b              0
hsc_p              0
hsc_b              0
hsc_s              0
degree_p           0
degree_t           0
workex             0
etest_p            0
specialisation     0
mba_p              0
status             0
salary            67
dtype: int64

In [4]:
df["salary"].describe()

count       148.000000
mean     288655.405405
std       93457.452420
min      200000.000000
25%      240000.000000
50%      265000.000000
75%      300000.000000
max      940000.000000
Name: salary, dtype: float64

In [5]:
m=round(df["salary"].mean(),0)
df["salary"].replace(np.nan,m,inplace=True)
df.drop("sl_no",axis=1,inplace=True)

In [6]:
df["gender"].value_counts()
dic={"M":0,"F":1}
df["gender"]=df["gender"].map(dic)

In [7]:
df["ssc_b"].value_counts()
dic={"Others":0,"Central":1}
df["ssc_b"]=df["ssc_b"].map(dic)

In [8]:
df["hsc_b"].value_counts()
dic={"Others":0,"Central":1}
df["hsc_b"]=df["hsc_b"].map(dic)
df["workex"].value_counts()
dic={"No":0,"Yes":1}
df["workex"]=df["workex"].map(dic)
df["specialisation"].value_counts()
dic={"Mkt&Fin":0,"Mkt&HR":1}
df["specialisation"]=df["specialisation"].map(dic)
df["status"].value_counts()
dic={"Placed":1,"Not Placed":0}
df["status"]=df["status"].map(dic)
df["degree_t"]=df["degree_t"].astype("object")
df=pd.get_dummies(df,drop_first=True)
df["s_d"]=df["ssc_p"]*df["degree_p"]
df["h_d"]=df["hsc_p"]*df["degree_p"]

I have created three models:

1. ssc_p and degree_p as independent variable while mba_p as dependent variable
2. hsc_p and degree_p as independent variable while mba_p as dependent variable
3. ssc_p, hsc_p and degree_p as independent variable while mba_p as dependent variable

# **MODEL1**

In [9]:
model1=ols("mba_p~ssc_p+degree_p+s_d",data=df).fit()
print(model1.summary())

                            OLS Regression Results                            
Dep. Variable:                  mba_p   R-squared:                       0.236
Model:                            OLS   Adj. R-squared:                  0.226
Method:                 Least Squares   F-statistic:                     21.78
Date:                Tue, 05 May 2020   Prob (F-statistic):           2.49e-12
Time:                        21:02:09   Log-Likelihood:                -654.74
No. Observations:                 215   AIC:                             1317.
Df Residuals:                     211   BIC:                             1331.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     96.6588     19.302      5.008      0.0

In [10]:
anova_table=sm.stats.anova_lm(model1,type=1)
print(anova_table)

             df       sum_sq      mean_sq          F        PR(>F)
ssc_p       1.0  1098.972558  1098.972558  41.705812  7.188130e-10
degree_p    1.0   382.792427   382.792427  14.526904  1.813008e-04
s_d         1.0   240.333951   240.333951   9.120631  2.839317e-03
Residual  211.0  5559.973456    26.350585        NaN           NaN


# **MODEL2**

In [11]:
model2=ols("mba_p~hsc_p+h_d",data=df).fit()
print(model2.summary())

                            OLS Regression Results                            
Dep. Variable:                  mba_p   R-squared:                       0.217
Model:                            OLS   Adj. R-squared:                  0.210
Method:                 Least Squares   F-statistic:                     29.40
Date:                Tue, 05 May 2020   Prob (F-statistic):           5.37e-12
Time:                        21:02:09   Log-Likelihood:                -657.43
No. Observations:                 215   AIC:                             1321.
Df Residuals:                     212   BIC:                             1331.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     54.6523      2.405     22.729      0.0

In [12]:
anova_table=sm.stats.anova_lm(model2,type=1)
print(anova_table)

             df       sum_sq     mean_sq          F        PR(>F)
hsc_p       1.0   916.806171  916.806171  34.094071  1.952231e-08
h_d         1.0   664.482964  664.482964  24.710708  1.372397e-06
Residual  212.0  5700.783258   26.890487        NaN           NaN


# **MODEL3**

In [13]:
model3=ols("mba_p~ssc_p+degree_p+s_d+h_d",data=df).fit()
print(model3.summary())

                            OLS Regression Results                            
Dep. Variable:                  mba_p   R-squared:                       0.266
Model:                            OLS   Adj. R-squared:                  0.252
Method:                 Least Squares   F-statistic:                     19.06
Date:                Tue, 05 May 2020   Prob (F-statistic):           2.17e-13
Time:                        21:02:09   Log-Likelihood:                -650.44
No. Observations:                 215   AIC:                             1311.
Df Residuals:                     210   BIC:                             1328.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    105.9095     19.227      5.508      0.0

In [14]:
anova_table=sm.stats.anova_lm(model3,type=1)
print(anova_table)

             df       sum_sq      mean_sq          F        PR(>F)
ssc_p       1.0  1098.972558  1098.972558  43.200370  3.838173e-10
degree_p    1.0   382.792427   382.792427  15.047486  1.402743e-04
s_d         1.0   240.333951   240.333951   9.447475  2.394858e-03
h_d         1.0   217.791472   217.791472   8.561335  3.811440e-03
Residual  210.0  5342.181984    25.438962        NaN           NaN


# **LOGISTIC REGRESSION**

In [15]:
col=["status","salary","degree_t_Others","ssc_b","hsc_b"]

data=df.drop(col,axis=1)
x=data.values
y=df["status"].values
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=1001)
lgr=LogisticRegression()
model_fit=lgr.fit(x_train,y_train)
prediction=lgr.predict(x_test)
print(accuracy_score(y_test,prediction))
print(confusion_matrix(y_test,prediction))

0.9302325581395349
[[ 8  2]
 [ 1 32]]
